In [3]:
import pandas as pd

In [13]:
data = pd.read_csv('stackoverflow_reduced.csv')

In [14]:
data.head()

,post,tags
0,trying to calculate and print the mean and its...,python
1,identifying server timeout quickly in iphone ...,iphone
2,distance between 2 or more drop pins i was do...,iphone
3,sql query - how to exclude a record from anoth...,sql
4,complicated min/max multi-table query i need ...,sql


In [15]:
print("Input data has {} rows and {} columns".format(len(data), len(data.columns)))

Input data has 8000 rows and 2 columns


In [16]:
print(data['post'].apply(lambda x: len(x.split(' '))).sum())

1625547


In [11]:
import nltk
import pandas as pd
import re
import string

In [12]:
stopwords = nltk.corpus.stopwords.words('english')
wn = nltk.WordNetLemmatizer()

In [17]:
def clean_text(text):
    text = "".join([word.lower() for word in text if word not in string.punctuation])
    tokens = re.split('\W+', text)
    text = [wn.lemmatize(word) for word in tokens if word not in stopwords]
    return text

In [19]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

In [21]:
count_vect = CountVectorizer(analyzer=clean_text)
X_count = count_vect.fit_transform(data['post'])
X_count_feat = pd.DataFrame(X_count.toarray())
X_count_feat.head()

,0,1,2,3,4,5,6,7,8,9,...,77173,77174,77175,77176,77177,77178,77179,77180,77181,77182
0,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [22]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.model_selection import train_test_split

C:\Users\Swetha\Anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [23]:
X_train, X_test, y_train, y_test = train_test_split(X_count_feat, data['tags'], test_size=0.2)

In [24]:
def train_GB(est, max_depth, lr):
    gb = GradientBoostingClassifier(n_estimators=est, max_depth=max_depth, learning_rate=lr)
    gb_model = gb.fit(X_train, y_train)
    y_pred = gb_model.predict(X_test)
    precision, recall, fscore, train_support = score(y_test, y_pred)
    print('Est: {} / Depth: {} / LR: {} ---- Precision: {} / Recall: {} / Accuracy: {}'.format(
        est, max_depth, lr, round(precision, 3), round(recall, 3), 
        round((y_pred==y_test).sum()/len(y_pred), 3)))

In [ ]:
for n_est in [100, 150]:
    for max_depth in [11, 15]:
        for lr in [0.1]:
            train_GB(n_est, max_depth, lr)